# AirBNB Data Preparation (Target = price_gte_150)

In this notebook we focus on the data loading, basic exploring, and prepatation.

This notebook follows closely the previous data cleaning toturial from last week. We will be using the same input dataset and producing the same output with one small change - instead of the target being the continuous variable price, we will select the target price_category - and we will be using a logistic regression model instead of a linear regression model to identify where price_gte_150 is true.


## 1.0 Setup


In [1]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
# set random seed to ensure that results are repeatable
np.random.seed(1)

## 2.0 Load data 

In [2]:
# load data
airbnb = pd.read_csv("E:/Spring-23/DSP/airbnb.csv")

airbnb.head(3)

,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,cancellation_policy,price,price_gte_150,price_category
0,0,0,Roslindale,42.282619,-71.133068,House,Entire home/apt,4,1.5,2.0,...,1,0,2,0,0,NaN,moderate,250,1,gte_226
1,0,1,Roslindale,42.286241,-71.134374,Apartment,Private room,2,1.0,1.0,...,0,0,2,36,804,94.0,moderate,65,0,lte_$75
2,1,1,Roslindale,42.292438,-71.135765,Apartment,Private room,2,1.0,1.0,...,1,20,3,41,2574,98.0,moderate,65,0,lte_$75


## 3.0 Conduct initial exploration of the data

We have a number of input variables and one target variable. For this analysis, the target variable is price.

First, our initial exploration of the data should answer the following questions:
1. How many rows and columns
2. How much of a problem do we have with na's?
3. What types of data are there?
4. What types of data are stored in columns
    1. identify which variables are numeric and may need to be standardized later
    2. identify which variables are categorical and may need to be transformed using and encoders such as one-hot-encoder.
5. Identify errors in the data - this is a common problem with categorical vars where the category is mispelled or spelled differently in some instances.
 

In [3]:
# look at the data
airbnb.head(3) # note that we don't want to dump all the data to the screen

,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,...,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,cancellation_policy,price,price_gte_150,price_category
0,0,0,Roslindale,42.282619,-71.133068,House,Entire home/apt,4,1.5,2.0,...,1,0,2,0,0,NaN,moderate,250,1,gte_226
1,0,1,Roslindale,42.286241,-71.134374,Apartment,Private room,2,1.0,1.0,...,0,0,2,36,804,94.0,moderate,65,0,lte_$75
2,1,1,Roslindale,42.292438,-71.135765,Apartment,Private room,2,1.0,1.0,...,1,20,3,41,2574,98.0,moderate,65,0,lte_$75


In [4]:
# generate a basic summary of the data
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3555 entries, 0 to 3554
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   host_is_superhost                  3555 non-null   int64  
 1   host_identity_verified             3555 non-null   int64  
 2   neighbourhood_cleansed             3555 non-null   object 
 3   latitude                           3555 non-null   float64
 4   longitude                          3555 non-null   float64
 5   property_type                      3552 non-null   object 
 6   room_type                          3555 non-null   object 
 7   accommodates                       3555 non-null   int64  
 8   bathrooms                          3541 non-null   float64
 9   bedrooms                           3545 non-null   float64
 10  beds                               3546 non-null   float64
 11  bed_type                           3555 non-null   objec

In [5]:
# generate a statistical summary of the numeric value in the data
airbnb.describe()

,host_is_superhost,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,Number of amenities,guests_included,price_per_extra_person,minimum_nights,number_of_reviews,number_days_btw_first_last_review,review_scores_rating,price,price_gte_150
count,3555.000000,3555.000000,3555.000000,3555.000000,3555.000000,3541.000000,3545.000000,3546.000000,3555.00000,3555.000000,3555.000000,3555.000000,3555.000000,3555.000000,2755.000000,3555.000000,3555.000000
mean,0.113080,0.727989,42.339973,-71.084874,3.023629,1.215899,1.246544,1.597293,14.85879,1.427004,10.886639,3.116737,19.126582,279.052602,91.891470,166.060478,0.500422
std,0.316735,0.445058,0.024464,0.031614,1.754808,0.492656,0.738440,0.995467,4.82126,1.050204,19.092755,8.273949,35.666178,408.686952,9.548381,103.378456,0.500070
min,0.000000,0.000000,42.235942,-71.171789,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,20.000000,10.000000,0.000000
25%,0.000000,0.000000,42.329875,-71.105183,2.000000,1.000000,1.000000,1.000000,12.00000,1.000000,0.000000,1.000000,1.000000,0.000000,89.000000,85.000000,0.000000
50%,0.000000,1.000000,42.345191,-71.078487,2.000000,1.000000,1.000000,1.000000,15.00000,1.000000,0.000000,2.000000,5.000000,92.000000,94.000000,150.000000,1.000000
75%,0.000000,1.000000,42.354672,-71.062142,4.000000,1.000000,2.000000,2.000000,18.00000,1.000000,20.000000,3.000000,21.000000,402.000000,98.000000,219.000000,1.000000
max,1.000000,1.000000,42.389982,-71.000100,16.000000,6.000000,5.000000,16.000000,30.00000,14.000000,200.000000,300.000000,404.000000,2680.000000,100.000000,650.000000,1.000000


In [7]:
!pip install summarytools

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for summarytools: filename=summarytools-0.2.3-py3-none-any.whl size=8596 sha256=11e6ba2621a6b9f68f1ccad10f0fe1d4d98564e0259d9ac90c1e1bb65f2de43b
  Stored in directory: c:\users\mssur\appdata\local\pip\cache\wheels\ba\b9\0e\dca846354555f16ac3dd84d4b305072a30f33d6d4243c38bfa
Successfully built summarytools


In [8]:
# there are many ways we could explore our data. A rather new library available is called 
# jupyter-summarytools this library provides functions that provide very thorough summaries 
# of your data. Though such detail is not always required, there are times when you want a 
# thorough summary

# jupyter-summary tools is not part of the standard anaconda distribution of python, nor 
# is it in any conda channels. To install this library, you need to install it from the 
# terminal/command line using pip pip install jupyter-summarytools

# once installed, you can import this library and use dfSummary to provide a more thorough 
# summary of your data
import summarytools
from summarytools import dfSummary
dfSummary(airbnb)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,host_is_superhost[int64],Mean (sd) : 0.1 (0.3)min < med < max:0.0 < 0.0 < 1.0IQR (CV) : 0.0 (0.4),2 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAAA1klEQVR4nO3cuwkDMRAAUZ9xuLEucFWuQLWpAlflwIqVyxUIDk7n38xLBWJhYMNdeu8n/b/zpwfQe1xGDymlW0SsWz5prT1rrfd5Y2m2YeiIWHPOjy2flFKu80bSEVzdEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQxgawtAQhoYwNIShIQwNYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQxgaYnhQTt9l7yVHQ/+IvZccXd0QhoYwNIShIV7AyBvfAtHoMwAAAABJRU5ErkJggg=="">",0(0.0%)
2,host_identity_verified[int64],Mean (sd) : 0.7 (0.4)min < med < max:0.0 < 1.0 < 1.0IQR (CV) : 1.0 (1.6),2 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAAA10lEQVR4nO3bsQnDMBBA0SS4vFouPJUn0GyaIFOliGr1ShsCBoEhdvL/K8UVgg/X3bX3ftH/ux39AX3HdPQHNCaltEbEPDLbWnvWWu/vb4b+EREx55wfI7OllOXzzdUNYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQxgawtAQhoYwNIShIQwNYWgIQ0MYGmLzyG7v9Z7OZTP03us9nYurG8LQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQ7wAhVMb3/9AvTYAAAAASUVORK5CYII="">",0(0.0%)
3,neighbourhood_cleansed[object],1. Jamaica Plain2. South End3. Back Bay4. Fenway5. Dorchester6. Allston7. Beacon Hill8. Brighton9. Downtown10. South Boston11. other,"341 (9.6%)323 (9.1%)298 (8.4%)285 (8.0%)269 (7.6%)260 (7.3%)188 (5.3%)184 (5.2%)172 (4.8%)171 (4.8%)1,064 (29.9%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAG8AAACzCAYAAACHOUI0AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAACP0lEQVR4nO3dMWoCURhG0UxIOa1YZFVZgWtzBVlVilhPbzagzEPQvAvn1BbCha94vyTL9Xp9o+n9v78AjxMvTLww8cLECxMvTLyw5XA4fK3rehz58LZtv5fL5fvZX4oxH+u6Hk+n08/Ih8/n8+ezvxDjzGaYeGHihYkXJl6YeGHihYkXJl6YeGHeNsMWP0DqMpthw7NpMuczfBJyDpqP2QwTL0y8MPHCxAsTL0y8MPHCxAsTL8zbZpiTUJjZDBuaTZM5p6GTkHPQnMxmmHhh4oWJFyZemHhh4oWJFyZemHhh3jbDnITCzGaY2QxzEgozm2HihYkXJl6YeGHihYkXJl6YeGHihXnbDHMSCjObYbuzaTLntXsScg6al9kMEy9MvDDxwsQLEy9MvDDxwsQLEy/M22aYk1CY2Qwzm2FOQmFmM0y8MPHCxAsTL0y8MPHCxAsTL0y8MG+bYU5CYWYz7O5smsv53T0JOQXNz2yGiRcmXph4YeKFiRcmXph4YeKFiRfmbTPMSSjMbIaZzTAnoTCzGSZemHhh4oWJFyZemHhh4oWJFyZemHhhHqbD3PPCzGbYzdk0mQ0373lueQ1mM0y8MPHCxAsTL0y8MPHCxAsTL0y8MG+bYU5CYWYzzGyGOQmFmc0w8cLECxMvTLww8cLECxMvTLww8cK8bYY5CYWZzbDd/5/3Cmb6MXf/6t8rOUE9xmyGiRcmXph4YeKFiRcmXph4YeKFiRfmbTPMSSjMbIaJFyZemHhh4oWJF/YHiXQDjYc2LrkAAAAASUVORK5CYII="">",0(0.0%)
4,latitude[float64],Mean (sd) : 42.3 (0.0)min < med < max:42.2 < 42.3 < 42.4IQR (CV) : 0.0 (1730.7),"3,554 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAABOElEQVR4nO3bsW2EMBhAYRKdROO0XJGOBZIFMkQmoM1aniBTXRFqGiQkUkWX7vBxYOT3vprC0hM/IJuneZ4rle859wK0j1PuBZSkaZrPEMJ56fXDMPz0ff+95Zr+GPqBQgjnrusuS6+PMb5uuZ7/HN0QhoYwNIShIQwNYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hGfGbkg58DdN01tVVYvPjO3J0DekHPiLMX5svZ57ObohDA1haAhDQxgawtAQfl5lNI7je9u2X0uuXftDnqEzquv6JeEbfdUPeY5uCENDGBrC0BDIl7FSdqRSIEOXsiOVwtENYWiIYkY38bmbopjQxOduCkc3xKHvaMfx1doNkEOHdhxfrd0AcXRD7H5HO47z2D204ziPh4T2Lj2+X2cgYQmqJZVJAAAAAElFTkSuQmCC"">",0(0.0%)
5,longitude[float64],Mean (sd) : -71.1 (0.0)min < med < max:-71.2 < -71.1 < -71.0IQR (CV) : 0.0 (-2248.5),"3,554 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAABRElEQVR4nO3bsW2EMBhAYSeKROO0XJGOBZIFMkQmoM1aTJCpUoSaBgmJlKlOwQf4LL/31RRGT1i/LPOwrmtQ/R7vvQDl8XTvBZSubduPGONly7PTNP2M4/h19ppuYeh/xBgvfd9/b3l2GIaXs9dzK2TolK90WZbXEMKm0CVDhk78St/PXk8ODmMQhoYwNIShIQwNYWgIQ0MYGsLQEIaG

In [9]:
# Check the missing values by summing the total na's for each variable
airbnb.isna().sum()

host_is_superhost                      0
host_identity_verified                 0
neighbourhood_cleansed                 0
latitude                               0
longitude                              0
property_type                          3
room_type                              0
accommodates                           0
bathrooms                             14
bedrooms                              10
beds                                   9
bed_type                               0
Number of amenities                    0
guests_included                        0
price_per_extra_person                 0
minimum_nights                         0
number_of_reviews                      0
number_days_btw_first_last_review      0
review_scores_rating                 800
cancellation_policy                    0
price                                  0
price_gte_150                          0
price_category                         0
dtype: int64

In [10]:
# create a list of these catagorical variables
category_var_list = list(airbnb.select_dtypes(include='object').columns)
category_var_list

['neighbourhood_cleansed',
 'property_type',
 'room_type',
 'bed_type',
 'cancellation_policy',
 'price_category']

In [11]:
# explore the categorical variable values - often there are typos here that need to be fixed.
for cat in category_var_list: # generally, we want to avoid for loops and use a functional style (i.e. list comprehension)
    print(f"Category: {cat} Values: {airbnb[cat].unique()}")

Category: neighbourhood_cleansed Values: ['Roslindale' 'Jamaica Plain' 'Mission Hill' 'Longwood Medical Area'
 'Bay Village' 'Leather District' 'Chinatown' 'North End' 'Roxbury'
 'South End' 'Back Bay' 'East Boston' 'Charlestown' 'West End'
 'Beacon Hill' 'Downtown' 'Fenway' 'Brighton' 'West Roxbury' 'Hyde Park'
 'Mattapan' 'Dorchester' 'South Boston Waterfront' 'South Boston'
 'Allston']
Category: property_type Values: ['House' 'Apartment' 'Condominium' 'Villa' 'Bed & Breakfast' 'Townhouse'
 'Entire Floor' 'Loft' 'Guesthouse' 'Boat' 'Dorm' 'Other' nan 'Camper/RV']
Category: room_type Values: ['Entire home/apt' 'Private room' 'Shared room']
Category: bed_type Values: ['Real Bed' 'Pull-out Sofa' 'Futon' 'Airbed' 'Couch']
Category: cancellation_policy Values: ['moderate' 'flexible' 'strict' 'super_strict_30']
Category: price_category Values: ['gte_226' 'lte_$75' 'btw_$75-$150' 'btw_$151-$225']


### Summary the findings from our initial evaluation of the data

* We have 6 categorical variables
* We have 3 variables that have missing values
* There doesn't seem to be a problem with the catogorical class names.

## 4.0 Process the data

* Conduct any data prepartion that should be done *BEFORE* the data split.
* Split the data.
* Conduct any data preparation that should be done *AFTER* the data split.

### 4.1  Conduct any data prepartion that should be done *BEFORE* the data split

Tasks at this stage include:
1. Drop any columns/features 
2. Decide if you with to exclude any observations (rows) due to missing na's.
2. Conduct proper encoding of categorical variables
    1. You can transform them using dummy variable encoding, one-hot-encoding, or label encoding. 

#### Drop any columns/variables we will not be using

In [12]:
# Our target is price; but there are three related price variableds - price, price_gte_150, 
# and price_category. We need to drop price_gte_150, and price_category
airbnb.drop(['price', 'price_category'], axis=1, inplace = True) # keep price_gte_150 for our target

#### Drop observations with too many NA's

If we want to remove the rows with NA's use the following code that is commented out. For this exercise - we will not drop rows with NA's 

In [13]:
# If we want to remove rows with NA's use the following code:
# airbnb.dropna(axis=0, inplace=True)

In [14]:
# verify that there are now no missing values
# airbnb.isna().sum()

In [15]:
# investigage how many rows remain 
# airbnb.shape

#### Encode our categorical variables

Categorical variables usually have strings for their values. Many machine learning algorithms do not support string values for the input variables. Therefore, we need to replace these string values with numbers. This process is called categorical variable encoding.

In a previous step we identified 5 catagorical variables and found no indication of typos in the class names. Our focus is now on encoding the variables. 

We have three main approaches to encoding variables (these will be discussed in greater detail in class)
* One-Hot-Encoding
* Dummy Encoding
* Label Encoding

In this exercise; we will dummy encode neighbourhood_cleansed, property_type using dummy encoding, and room_type, bed_type and cancelation policy using label encoding. (we will have more discussion on these choices in class).

Before we do our encoding, we must identify if any of our categorical variables have a missing value. We will replace any missing values with the term 'unkown'.

In [16]:
airbnb['property_type'].isna().sum() # check for missing values in this variable/column - we can see there are three for this variable

3

In [17]:
airbnb["property_type"].fillna("unkown", inplace = True)

In [18]:
airbnb['neighbourhood_cleansed'].isna().sum() 

0

In [19]:
airbnb['room_type'].isna().sum() # can see by the results below, no missing values

0

In [20]:
airbnb['bed_type'].isna().sum() # can see by the results below, no missing values

0

In [21]:
airbnb['cancellation_policy'].isna().sum() # can see by the results below, no missing values

0

Now, let's encode neighborhood_cleansed and property_type as dummy variables and room_type, bed_type and cancelation_policy labeled (numeric)

In [22]:
dummies_df = pd.get_dummies(airbnb['neighbourhood_cleansed'], prefix='neighbourhood_cleansed', drop_first=True)

In [23]:
airbnb = airbnb.join(dummies_df)
airbnb.drop('neighbourhood_cleansed', axis=1, inplace = True)

In [24]:
airbnb = airbnb.join(pd.get_dummies(airbnb['property_type'], prefix='property_type', drop_first=True))
airbnb.drop('property_type', axis=1, inplace = True)

In [25]:
labelencoder = LabelEncoder()
airbnb['room_type'] = labelencoder.fit_transform(airbnb['room_type'])
airbnb['bed_type'] = labelencoder.fit_transform(airbnb['bed_type'])
airbnb['cancellation_policy'] = labelencoder.fit_transform(airbnb['cancellation_policy'])

In [26]:
# explore the dataframe columns to verify encoding and dropped columns
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3555 entries, 0 to 3554
Data columns (total 56 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   host_is_superhost                               3555 non-null   int64  
 1   host_identity_verified                          3555 non-null   int64  
 2   latitude                                        3555 non-null   float64
 3   longitude                                       3555 non-null   float64
 4   room_type                                       3555 non-null   int32  
 5   accommodates                                    3555 non-null   int64  
 6   bathrooms                                       3541 non-null   float64
 7   bedrooms                                        3545 non-null   float64
 8   beds                                            3546 non-null   float64
 9   bed_type                                 

### 4.2 Split data (train/test)

In [27]:
# split the data into validation and training set
train_df, test_df = train_test_split(airbnb, test_size=0.3)

# to reduce repetition in later code, create variables to represent the columns
# that are our predictors and target
target = 'price_gte_150'
predictors = list(airbnb.columns)
predictors.remove(target)

### 4.3  Conduct any data prepartion that should be done *AFTER* the data split

We will look at the following:
1) imput any missing numeric values using the mean of the variable/column
2) remove differences of scale by standardizing the numerica variables

#### Impute missing values

In [28]:
numeric_cols_with_nas = list(train_df.isna().sum()[train_df.isna().sum() > 0].index)
numeric_cols_with_nas

['bathrooms', 'bedrooms', 'beds', 'review_scores_rating']

We can see from the code above that there are 4 variables (columns) that contain missing numeric values (we've already taken care of any missing values in the catagorical variables earlier).

In [29]:
imputer = SimpleImputer(strategy="median")

train_df[numeric_cols_with_nas] = imputer.fit_transform(train_df[numeric_cols_with_nas])
test_df[numeric_cols_with_nas] = imputer.transform(test_df[numeric_cols_with_nas])

#### Standardize numeric values

Now, let's create a common scale between the numberic columns by standardizing each numeric column

In [30]:
# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
cols_to_stdize = ['latitude', 'longitude', 'accommodates', 
                   'bathrooms', 'bedrooms', 'beds', 'Number of amenities', 
                   'guests_included', 'price_per_extra_person', 'minimum_nights', 
                   'number_of_reviews', 'number_days_btw_first_last_review', 
                   'review_scores_rating']                
               
# Transform the predictors of training and validation sets
train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) # train_predictors is not a numpy array


test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) # validation_target is now a series object


## 5.0 Save the data

In [32]:
train_X = train_df[predictors]
train_y = train_df[target] # train_target is now a series objecttrain_df.to_csv('airbnb_train_df.csv', index=False)
test_X = test_df[predictors]
test_y = test_df[target] # validation_target is now a series object

train_df.to_csv('E:/Spring-23/DSP/airbnb_train_df_price_gte_150.csv', index=False)
train_X.to_csv('E:/Spring-23/DSP/airbnb_train_X_price_gte_150.csv', index=False)
train_y.to_csv('E:/Spring-23/DSP/airbnb_train_y_price_gte_150.csv', index=False)
test_df.to_csv('E:/Spring-23/DSP/airbnb_test_df_price_gte_150.csv', index=False)
test_X.to_csv('E:/Spring-23/DSP/airbnb_test_X_price_gte_150.csv', index=False)
test_y.to_csv('E:/Spring-23/DSP/airbnb_test_y_price_gte_150.csv', index=False)